In [10]:
!pip install pyrankvote==2.0.0

  Attempting uninstall: pyrankvote
    Found existing installation: pyrankvote 1.0.9
    Uninstalling pyrankvote-1.0.9:
      Successfully uninstalled pyrankvote-1.0.9


In [11]:
from platform import python_version

print(python_version())

3.7.6


In [12]:
import pyrankvote
import ast
import pandas as pd
from pyrankvote import Candidate, Ballot


def rename_index_col_to_ballot_id(df):
    df.rename(columns={'Unnamed: 0': 'ballot_id'}, inplace=True)
    return df

def get_cand_list(df):
    list_of_unique_vals_in_dataframe = df.iloc[:, 1].value_counts().index
    cand_list = list(filter(lambda x: x != '0', list_of_unique_vals_in_dataframe))
    return cand_list

def initialize_cand_objs(cand_list):
    if isinstance(cand_list, str):
        return [Candidate(c) for c in cand_list.split(', ')]
    else:
        return [Candidate(c) for c in cand_list]

# def initialize_cand_objs_in_df_cols(df):
#     cands_df = df.iloc[:, 1:]  # getting all columns with candidate names in them
#     cands_df = cands_df.applymap(lambda x: create_cand(x))  # making them into pyrankvote candidate objects
#     cands_df['ballot_id'] = df.iloc[:, 0]  # stitching df back with ballot_ids from old dataframe
#     return cands_df

def get_cands_into_single_cell(df):
    df['candidate_list'] = df.iloc[:, 1:].agg(", ".join, axis=1)
    # cands_to_put_in_single_cell = df.iloc[:, 1:].columns.to_list()
    # df['candidate_list'] = str(cands_to_put_in_single_cell)
    # df['candidate_list'] = df['candidate_list'].apply(lambda x: ast.literal_eval(x))
    return df

def initialize_ballot_objs(df):
    ballot_objects = []
    for index, value in enumerate(df['candidate_list']):
        ballot = Ballot(ranked_candidates=value)
        ballot_objects.append(ballot)
    return ballot_objects

def run_election(list_of_cand_objs, election_df):
    return pyrankvote.instant_runoff_voting(list_of_cand_objs, election_df['ballots'], pick_random_if_blank=True)

def rm_invalid_rows(df):
    return df[df['candidate_list'] != '0']

def run_all_steps(df):
    election_dict = {}
    #df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
#     df = rename_index_col_to_ballot_id(df)
    df = df.dropna(axis=1, how='all')
    df = df.drop(['ballot_id','filename'], axis=1)
    print('df.shape:',df.shape)
    df = get_cands_into_single_cell(df)

    df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ', '').replace(', 0', ''))
    df = rm_invalid_rows(df)
    df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))

    ballots = initialize_ballot_objs(df)
    df['ballots'] = ballots
    
    cand_list = get_cand_list(df)
    cand_list = initialize_cand_objs(cand_list)
    print("cand_list: ",cand_list)
#     print(df.head())
    election = run_election(cand_list, df)
    winner = election.get_winners()
    print("get_winners",winner)
    election_dict['winner'] = winner
    #df["irv_winner"] = winner
    print("rounds",len(election.rounds))
    election_dict['rounds'] = len(election.rounds)

#     open file and write results. read winners votes from file and write to df
    #print("register_round_results",election.register_round_results(election.rounds[-1]))

    print("election:", election)
    #election_dict['election'] = election

#     print("election type:", type(election))
    #election_dict['register']=election.register_round_results(election.rounds[-1])

    #print('hi')
    return df, election_dict

def read_votes(file, election_dict):
    with open(file, 'r') as file1:
        count = 0
        while True: 
            count += 1

            # Get next line from file 
            line = file1.readline()
            if not line: 
                break
            strip_line = line.strip()
    #         print('*',strip_line,'*')
            if 'ROUND' in strip_line or 'FINAL' in strip_line:
                start_round = True
                if 'ROUND 1' in strip_line or 'FINAL' in strip_line:
                    monitor_round=True
                    print('*',strip_line,'*')
                else:
                    monitor_round=False
            if 'ROUND' not in strip_line and  'FINAL' not in strip_line:
                start_round = False
                if winner in line and monitor_round:
                    print('*',strip_line,'*')
                    if 'firstRoundVotes' not in election_dict:
                        election_dict['firstRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                    else:
                        election_dict['finalRoundVotes']=[int(s) for s in strip_line.split() if s.isdigit()].pop()
                    
    file1.close() 
    

In [13]:
df = pd.read_csv('pyrankobjs/master_elections.csv')
df = rename_index_col_to_ballot_id(df)
# df['irv_winner']=""
df = df[(df['filename']!="election_05-04-2020_08-39-40_5cands_12noise.csv") | (df['ballot_id']!=2767)]
df = df[(df['filename']!="election_04-30-2020_21-07-57_4_0.01.csv") | (df['ballot_id']!=20726)]
master_df = df.copy(deep=True)

In [14]:
election_dict = {}
#df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
#     df = rename_index_col_to_ballot_id(df)
df = df[df['filename']=="election_05-04-2020_08-39-40_5cands_12noise.csv"]
df.to_csv('election11.csv', index = False, header=True)
df = df.dropna(axis=1, how='all')
df = df.drop(['ballot_id','filename'], axis=1)
print('df.shape:',df.shape)
df = get_cands_into_single_cell(df)

df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ', '').replace(', 0', ''))
df = rm_invalid_rows(df)
df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))

ballots = initialize_ballot_objs(df)
df['ballots'] = ballots

cand_list = get_cand_list(df)
cand_list = initialize_cand_objs(cand_list)
print("cand_list: ",cand_list)
#     print(df.head())
election = run_election(cand_list, df)
winner = election.get_winners()
print("get_winners",winner)
election_dict['winner'] = winner
#df["irv_winner"] = winner
print("rounds",len(election.rounds))
election_dict['rounds'] = len(election.rounds)

#     open file and write results. read winners votes from file and write to df
#print("register_round_results",election.register_round_results(election.rounds[-1]))

print("election:", election)
election_dict['election'] = election

#     print("election type:", type(election))
#election_dict['register']=election.register_round_results(election.rounds[-1])

#print('hi')
# return df, election_dict

df.shape: (2767, 5)
cand_list:  [<Candidate('candidate_1')>, <Candidate('candidate_4')>, <Candidate('candidate_3')>, <Candidate('candidate_2')>, <Candidate('candidate_5')>]


TypeError: instant_runoff_voting() got an unexpected keyword argument 'pick_random_if_blank'